In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
#ws = Workspace.get(name="quick-starts-ws-234286")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-235851
Azure region: westeurope
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-235851


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

cluster_name = "project-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target= ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
### YOUR CODE HERE ###

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C':choice(0.01,0.1,1,10,20,50,100,200,500), '--max_iter':choice(50,100,200)
    }
    )

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.15,delay_evaluation=5)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')


# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig("./", "train.py",  environment=sklearn_env, compute_target=compute_target)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps,
                                    primary_metric_name = 'Accuracy',
                                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4,
                                    policy=policy,
                                    run_config=src,
                                    )

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

#hyperdrive_run.get_status()

hyperdrive_run.wait_for_completion(show_output=True)
### YOUR CODE HERE ###ins

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_54715de9-9d00-45bb-965b-bed25e86788d
Web View: https://ml.azure.com/runs/HD_54715de9-9d00-45bb-965b-bed25e86788d?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-235851/workspaces/quick-starts-ws-235851&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-06-16T09:09:55.753428][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-06-16T09:09:56.3399818Z][SCHEDULER][INFO]Scheduling job, id='HD_54715de9-9d00-45bb-965b-bed25e86788d_0' 
[2023-06-16T09:09:56.4216970Z][SCHEDULER][INFO]Scheduling job, id='HD_54715de9-9d00-45bb-965b-bed25e86788d_1' 
[2023-06-16T09:09:56.5212554Z][SCHEDULER][INFO]Scheduling job, id='HD_54715de9-9d00-45bb-965b-bed25e86788d_2' 
[2023-06-16T09:09:56.594559][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-06-16T09:09:56.6420516Z][SCHEDULER][INFO]Scheduling job, id='HD_54715de9-9d00-45bb-965b-bed25e8678

{'runId': 'HD_54715de9-9d00-45bb-965b-bed25e86788d',
 'target': 'project-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-06-16T09:09:55.067626Z',
 'endTimeUtc': '2023-06-16T09:20:31.908711Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '74c489d7-b945-410a-9d0b-ad43927d0b57',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': '27',
  'score': '0.9110773899848255',
  'best_child_run_id': 'HD_54715de9-9d00-45bb-965b-bed25e86788d_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_54715de9-9d00-45bb-965b-bed25e86788d_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCli

In [6]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

### YOUR CODE HERE ###


print("best run details :", best_run.get_details())
print('\n')
print("best run file names :", best_run.get_file_names())
print('\n')
print("best run metrics :", best_run.get_metrics())



best run details : {'runId': 'HD_54715de9-9d00-45bb-965b-bed25e86788d_1', 'target': 'project-cluster', 'status': 'Completed', 'startTimeUtc': '2023-06-16T09:14:03.540332Z', 'endTimeUtc': '2023-06-16T09:15:17.965822Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': '74c489d7-b945-410a-9d0b-ad43927d0b57', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '20', '--max_iter', '50'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'project-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name': 'skl

In [7]:
model = best_run.register_model(model_name='hype_model', model_path="./")

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

tbds = TabularDatasetFactory.from_delimited_files(url)

In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=compute_target,
    experiment_timeout_minutes=30,
    task='classification', 
    primary_metric='accuracy',
    training_data=tbds,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=2)

In [10]:
# Submit your automl run
automl_run = exp.submit(config=automl_config, show_output=True)
automl_run.wait_for_completion()

### YOUR CODE HERE ###

Submitting remote run.
No run_configuration provided, running on project-cluster with default configuration
Running on remote compute: project-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_cf80b855-ed67-4285-82c5-c871fd39b838,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

{'runId': 'AutoML_cf80b855-ed67-4285-82c5-c871fd39b838',
 'target': 'project-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-06-16T09:21:41.611761Z',
 'endTimeUtc': '2023-06-16T09:57:31.092104Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'project-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-235851","workspace_name":"quick-starts-ws-235851","region":"westeurope","compute_target":"project-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_s

In [14]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

from azureml.automl.runtime.onnx_convert import OnnxConverter

automl_best_run_onnx, automl_fitted_model_onnx = automl_run.get_output(return_onnx_model=True)
print(automl_best_run_onnx.get_metrics())
OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './/best_model.onnx' )


{'norm_macro_recall': 0.5505962950859472, 'precision_score_macro': 0.7953594490820823, 'recall_score_weighted': 0.9172382397572079, 'average_precision_score_micro': 0.9812646237605938, 'f1_score_macro': 0.7848125657550904, 'recall_score_macro': 0.7752981475429737, 'precision_score_weighted': 0.9146094771404992, 'precision_score_micro': 0.9172382397572079, 'AUC_weighted': 0.9470965844710013, 'log_loss': 0.25839807118410246, 'recall_score_micro': 0.9172382397572079, 'matthews_correlation': 0.5702957677961882, 'f1_score_weighted': 0.9157906976996932, 'average_precision_score_macro': 0.8243929417932103, 'accuracy': 0.9172382397572079, 'AUC_micro': 0.9805143637414486, 'average_precision_score_weighted': 0.9552535233685324, 'weighted_accuracy': 0.9524987760420578, 'AUC_macro': 0.9470965844710014, 'balanced_accuracy': 0.7752981475429737, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_cf80b855-ed67-4285-82c5-c871fd39b838_28/accuracy_table', 'f1_score_micro': 0.9172382397572079, 

In [15]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

